# Anime Dataset Preprocessing


This part of the notebook cleans the **Anime Recommendation** dataset to prepare it for NLP-based recommendations.
It follows the proposal steps: normalize text (stopword removal + lemmatization) for *synopsis/plot* and clean/standardize the *genre* field; coerce numeric fields (*episodes*, *score*); and remove duplicates/missing-critical rows.


In [ ]:
# %% [setup]
import pandas as pd
import numpy as np
import re
import os
from pathlib import Path

# Text processing
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Ensure NLTK data is available
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)

DATASET_PATH = "/content/anime_recommendation_dataset.csv"
OUTPUT_CSV   = "/mnt/data/cleaned_anime_dataset.csv"

In [ ]:
# %% [load]
df = pd.read_csv(DATASET_PATH)
print("Shape:", df.shape)
print("Columns:", list(df.columns))
df.head(3)

Shape: (200, 6)
Columns: ['title', 'synopsis', 'genres', 'episodes', 'score', 'characters']


title  \
0                     Cowboy Bebop   
1  Cowboy Bebop: Tengoku no Tobira   
2                           TRIGUN   

                                            synopsis  \
0  Enter a world in the distant future, where Bou...   
1  As the Cowboy Bebop crew travels the stars, th...   
2  Vash the Stampede is a wanted man with a habit...   

                                     genres  episodes  score  \
0          Action, Adventure, Drama, Sci-Fi      26.0     86   
1            Action, Drama, Mystery, Sci-Fi       1.0     82   
2  Action, Adventure, Comedy, Drama, Sci-Fi      26.0     80   

                                          characters  
0  Spike Spiegel, Ein, Faye Valentine, Rhint Celo...  
1  Spike Spiegel, Ein, Faye Valentine, Lee Samson...  
2  Kuroneko, Milly Thompson, Vash the Stampede, R...

In [ ]:
# %% [resolve_columns]
def resolve_col(df, candidates):
    cols = {c.lower(): c for c in df.columns}
    # exact
    for cand in candidates:
        if cand.lower() in cols:
            return cols[cand.lower()]
    # contains
    for c in df.columns:
        lc = c.lower()
        for cand in candidates:
            if cand.lower() in lc:
                return c
    return None

col_title     = resolve_col(df, ["title", "name"])
col_genre     = resolve_col(df, ["genre", "genres"])
col_synopsis  = resolve_col(df, ["synopsis", "plot", "description", "summary"])
col_chars     = resolve_col(df, ["characters", "cast", "people"])
col_episodes  = resolve_col(df, ["episodes", "eps", "num_episodes"])
col_score     = resolve_col(df, ["score", "rating", "rank", "average_score"])

print({
    "title": col_title, "genre": col_genre, "synopsis": col_synopsis,
    "characters": col_chars, "episodes": col_episodes, "score": col_score
})

{'title': 'title', 'genre': 'genres', 'synopsis': 'synopsis', 'characters': 'characters', 'episodes': 'episodes', 'score': 'score'}


In [ ]:
# %% [basic_cleaning]
required = [col_title, col_genre, col_synopsis]
required = [c for c in required if c is not None]
if required:
    before = len(df)
    df = df.dropna(subset=required)
    df = df[df[required].applymap(lambda x: str(x).strip() != "").all(axis=1)]
    print(f"Dropped {before - len(df)} rows missing required fields.")

if col_title:
    before = len(df)
    df = df.drop_duplicates(subset=[col_title])
    print(f"Dropped {before - len(df)} duplicate titles.")

Dropped 0 rows missing required fields.
Dropped 0 duplicate titles.


/tmp/ipython-input-3034923210.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df[df[required].applymap(lambda x: str(x).strip() != "").all(axis=1)]


In [ ]:
# %% [numerics]
if col_episodes:
    df[col_episodes] = pd.to_numeric(df[col_episodes], errors='coerce').fillna(0).astype(int)
if col_score:
    # keep as float; fill with column mean if missing
    df[col_score] = pd.to_numeric(df[col_score], errors='coerce')
    if df[col_score].notna().any():
        mean_score = df[col_score].mean()
        df[col_score] = df[col_score].fillna(mean_score)
    else:
        # if all NaN, set to 0
        df[col_score] = 0.0

In [ ]:
# %% [text_helpers]
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text: str) -> str:
    text = re.sub(r'[^a-zA-Z\s]', ' ', str(text))
    tokens = []
    for w in text.split():
        lw = w.lower()
        if lw not in stop_words:
            tokens.append(lemmatizer.lemmatize(lw))
    return " ".join(tokens)

In [ ]:
# %% [clean_synopsis]
if True and col_synopsis is not None:
    df['clean_synopsis'] = df[col_synopsis].apply(clean_text)
else:
    df['clean_synopsis'] = ""  # fallback
df['clean_synopsis'].head(3)

0    enter world distant future bounty hunter roam ...
1    cowboy bebop crew travel star learn largest bo...
2    vash stampede wanted man habit turning entire ...
Name: clean_synopsis, dtype: object

In [ ]:
# %% [clean_genres]
def clean_genre_field(s: str) -> str:
    s = str(s).lower()
    # keep letters, commas, and spaces; then normalize spaces
    s = re.sub(r'[^a-z,\s]', ' ', s)
    s = re.sub(r'\s+', ' ', s).strip()
    # also normalize comma spacing
    s = re.sub(r'\s*,\s*', ',', s)
    return s

def split_genres(s: str):
    if not s:
        return []
    return [g.strip() for g in s.split(',') if g.strip()]

if col_genre is not None:
    df['genre_clean'] = df[col_genre].apply(clean_genre_field)
    df['genre_tokens'] = df['genre_clean'].apply(split_genres)
else:
    df['genre_clean'] = ""
    df['genre_tokens'] = [[]]

df[['genre_clean', 'genre_tokens']].head(5)

genre_clean  \
0         action,adventure,drama,sci fi   
1           action,drama,mystery,sci fi   
2  action,adventure,comedy,drama,sci fi   
3     action,drama,mystery,supernatural   
4        adventure,fantasy,supernatural   

                                 genre_tokens  
0          [action, adventure, drama, sci fi]  
1            [action, drama, mystery, sci fi]  
2  [action, adventure, comedy, drama, sci fi]  
3      [action, drama, mystery, supernatural]  
4          [adventure, fantasy, supernatural]

In [ ]:
# %% [finalize_columns]
out_cols = []
if col_title: out_cols.append(col_title)
if col_genre: out_cols.append('genre_clean'); out_cols.append('genre_tokens')
out_cols.append('clean_synopsis')
if col_score: out_cols.append(col_score)
if col_episodes: out_cols.append(col_episodes)

# Rename standard output names
rename_map = {}
if col_title: rename_map[col_title] = 'title'
if col_score: rename_map[col_score] = 'score'
if col_episodes: rename_map[col_episodes] = 'episodes'

df_out = df[out_cols].rename(columns=rename_map)

print("Output columns:", list(df_out.columns))
df_out.head(5)

Output columns: ['title', 'genre_clean', 'genre_tokens', 'clean_synopsis', 'score', 'episodes']


title                           genre_clean  \
0                     Cowboy Bebop         action,adventure,drama,sci fi   
1  Cowboy Bebop: Tengoku no Tobira           action,drama,mystery,sci fi   
2                           TRIGUN  action,adventure,comedy,drama,sci fi   
3               Witch Hunter ROBIN     action,drama,mystery,supernatural   
4                   Bouken Ou Beet        adventure,fantasy,supernatural   

                                 genre_tokens  \
0          [action, adventure, drama, sci fi]   
1            [action, drama, mystery, sci fi]   
2  [action, adventure, comedy, drama, sci fi]   
3      [action, drama, mystery, supernatural]   
4          [adventure, fantasy, supernatural]   

                                      clean_synopsis  score  episodes  
0  enter world distant future bounty hunter roam ...     86        26  
1  cowboy bebop crew travel star learn largest bo...     82         1  
2  vash stampede wanted man habit turning entire ...     80        26  
3  robin sena powerful craft user drafted stnj gr...     68        26  
4  dark century people suffering rule devil vande...     65        52

In [ ]:
# %% [save]
OUTPUT_CSV   = "/content/anime_recommendation_dataset.csv"
df_out.to_csv(OUTPUT_CSV, index=False)
print("Saved cleaned dataset to:", OUTPUT_CSV)

Saved cleaned dataset to: /content/anime_recommendation_dataset.csv


### Preview of Cleaned Data


In [ ]:
# %% [preview]
df_out.head(500)

title  \
0                                     Cowboy Bebop   
1                  Cowboy Bebop: Tengoku no Tobira   
2                                           TRIGUN   
3                               Witch Hunter ROBIN   
4                                   Bouken Ou Beet   
..                                             ...   
195                          Kidou Senkan Nadesico   
196  Kidou Senkan Nadesico: The prince of darkness   
197                                Kuro no Danshou   
198                                    Mezzo Forte   
199                                          MEZZO   

                                genre_clean  \
0             action,adventure,drama,sci fi   
1               action,drama,mystery,sci fi   
2      action,adventure,comedy,drama,sci fi   
3         action,drama,mystery,supernatural   
4            adventure,fantasy,supernatural   
..                                      ...   
195      action,comedy,mecha,romance,sci fi   
196        action,comedy,drama,mecha,sci fi   
197                   hentai,horror,mystery   
198             action,comedy,hentai,sci fi   
199  action,adventure,comedy,mystery,sci fi   

                                     genre_tokens  \
0              [action, adventure, drama, sci fi]   
1                [action, drama, mystery, sci fi]   
2      [action, adventure, comedy, drama, sci fi]   
3          [action, drama, mystery, supernatural]   
4              [adventure, fantasy, supernatural]   
..                                            ...   
195      [action, comedy, mecha, romance, sci fi]   
196        [action, comedy, drama, mecha, sci fi]   
197                     [hentai, horror, mystery]   
198              [action, comedy, hentai, sci fi]   
199  [action, adventure, comedy, mystery, sci fi]   

                                        clean_synopsis  score  episodes  
0    enter world distant future bounty hunter roam ...     86        26  
1    cowboy bebop crew travel star learn largest bo...     82         1  
2    vash stampede wanted man habit turning entire ...     80        26  
3    robin sena powerful craft user drafted stnj gr...     68        26  
4    dark century people suffering rule devil vande...     65        52  
..                                                 ...    ...       ...  
195  akito want fight despite childhood spent anime...     73        26  
196  two year passed since end martian successor na...     64         1  
197  based erotic game abogado power oz club br br ...     45         4  
198  mikura member mercenary team asked kidnap base...     64         2  
199  mikura kurokawa harada member danger service a...     62        13  

[200 rows x 6 columns]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful